<a href="https://colab.research.google.com/github/ayumu0622/recommendWe/blob/main/makefeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# pip install k-means-constrained

In [ ]:
import requests
import json
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
from bs4 import BeautifulSoup
from urllib.request import urlopen
from tqdm import tqdm

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

ep = "https://api.wevoteusa.org/apis/v1/politiciansSyncOut"

In [ ]:
statedf = pd.read_csv("/content/drive/MyDrive/recommendEngine/code.csv")

In [ ]:
def createDF(df, statelist):
  for st in tqdm(statelist, desc="Processing"):
    params = {'state_code':st}
    res = requests.get(ep, params=params)
    ids = json.loads(res.text)
    df2 = pd.DataFrame(ids)
    df = pd.concat([df, df2])
  return df

In [ ]:
#get the data from models.py
df = createDF(pd.DataFrame(), statedf['code'].to_list())

Processing: 100%|██████████| 50/50 [01:16<00:00,  1.52s/it]


In [ ]:
df

,we_vote_id,ballotpedia_id,ballotpedia_politician_name,ballotpedia_politician_url,bioguide_id,birth_date,cspan_id,ctcl_uuid,date_last_updated,date_last_updated_from_candidate,facebook_url,facebook_url2,facebook_url3,facebook_url_is_broken,facebook_url2_is_broken,facebook_url3_is_broken,fec_id,first_name,full_name_assembled,gender,google_civic_candidate_name,google_civic_candidate_name2,google_civic_candidate_name3,govtrack_id,house_history_id,icpsr_id,instagram_followers_count,instagram_handle,is_battleground_race_2019,is_battleground_race_2020,is_battleground_race_2021,is_battleground_race_2022,is_battleground_race_2023,is_battleground_race_2024,is_battleground_race_2025,is_battleground_race_2026,last_name,linked_campaignx_we_vote_id,linkedin_url,lis_id,maplight_id,middle_name,opensecrets_id,political_party,politician_contact_form_url,politician_email_address,politician_email,politician_email2,politician_email3,politician_facebook_id,politician_googleplus_id,politician_name,politician_phone_number,politician_phone_number2,politician_phone_number3,politician_twitter_handle,politician_twitter_handle2,politician_twitter_handle3,politician_twitter_handle4,politician_twitter_handle5,politician_url,politician_url2,politician_url3,politician_url4,politician_url5,politician_youtube_id,profile_image_type_currently_active,seo_friendly_path,seo_friendly_path_date_last_updated,state_code,thomas_id,twitter_description,twitter_followers_count,twitter_handle_updates_failing,twitter_handle2_updates_failing,twitter_location,twitter_name,twitter_profile_image_url_https,twitter_profile_background_image_url_https,twitter_profile_banner_url_https,twitter_user_id,vote_smart_id,vote_usa_politician_id,vote_usa_profile_image_url_https,washington_post_id,we_vote_hosted_profile_facebook_image_url_large,we_vote_hosted_profile_facebook_image_url_medium,we_vote_hosted_profile_facebook_image_url_tiny,we_vote_hosted_profile_image_url_large,we_vote_hosted_profile_image_url_medium,we_vote_hosted_profile_image_url_tiny,we_vote_hosted_profile_twitter_image_url_large,we_vote_hosted_profile_twitter_image_url_medium,we_vote_hosted_profile_twitter_image_url_tiny,we_vote_hosted_profile_uploaded_image_url_large,we_vote_hosted_profile_uploaded_image_url_medium,we_vote_hosted_profile_uploaded_image_url_tiny,we_vote_hosted_profile_vote_usa_image_url_large,we_vote_hosted_profile_vote_usa_image_url_medium,we_vote_hosted_profile_vote_usa_image_url_tiny,wikipedia_id,wikipedia_url,youtube_url
0,wv87pol78432,None,,,None,None,None,None,2023-08-07 15:26:48,None,https://www.facebook.com/people/Oronde-K-Mitch...,,,False,False,False,None,,None,M,Oronde Mitchell,Oronde K. Mitchell,,None,None,None,NaN,,False,False,False,False,False,False,False,False,,wv87camp74055,,None,None,,None,NON_PARTISAN,,None,,,,None,None,Oronde Mitchell,,,,None,None,None,None,None,,,,,,None,UPLOADED,oronde-mitchell-politician,None,AL,None,None,0,False,False,None,None,None,None,None,NaN,,,None,None,None,None,None,https://wevote-images.s3.amazonaws.com/wv87pol...,https://wevote-images.s3.amazonaws.com/wv87pol...,https://wevote-images.s3.amazonaws.com/wv87pol...,None,None,None,https://wevote-images.s3.amazonaws.com/wv87pol...,https://wevote-images.s3.amazonaws.com/wv87pol...,https://wevote-images.s3.amazonaws.com/wv87pol...,None,None,None,None,,
1,wv87pol13512,None,,,None,None,None,None,2023-01-26 22:26:13,2023-01-26 08:00:00,,None,None,False,False,False,None,Linda,None,U,Linda Riggins Aaron,None,None,None,None,None,NaN,,False,False,False,False,False,False,False,False,Aaron,wv87camp8549,None,None,None,Riggins,None,,,,None,None,None,None,None,Linda Riggins Aaron,,None,None,None,None,None,None,None,,None,None,None,None,None,UNKNOWN,linda-riggins-aaron-politician-from-alabama,None,al,None,None,0,False,False,None,None,None,None,None,NaN,None,ALAaronLindaRiggins,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,wv87pol50829,None,None,None,None,None,None,None,2023-0

In [ ]:
df.political_party = df.political_party.str.lower()
df.state_code = df.state_code.str.lower()

In [ ]:
statedf = statedf.rename(columns={'code':'state_code'})

In [ ]:
df = pd.merge(df, statedf, on='state_code', how='left')

In [ ]:
all_we_need = ['we_vote_id', 'political_party', 'politician_url', 'state_code', 'twitter_followers_count', 'twitter_description', 'image_flag', 'facebook_flag', 'google_civic_flag', 'phone_flag', 'lon', 'lat']

In [ ]:
def pExist(df, column):
  return 1 - df[column].isnull().sum() / df.shape[0]

In [ ]:
def replacer(value):
  if value == None:
    return 0
  else:
    return 1

In [ ]:
def makeFlag(df, keyword, delete_keyword, delete=True):
  image_have = df.loc[:, df.columns.str.startswith(keyword)]
  image_flag = image_have.applymap(replacer).iloc[:, 0]
  if delete:
    df = df.loc[:, ~df.columns.str.contains(delete_keyword, case=False)]
  df[f'{delete_keyword}_flag'] = image_flag
  return df

In [ ]:
df = makeFlag(df, 'we_vote_hosted_profile_image_url_', 'image')
df = makeFlag(df, 'facebook_url', 'facebook')
df = makeFlag(df, 'politician_phone', 'phone')
df = makeFlag(df, 'google_civic_', 'google_civic')
df = makeFlag(df, 'twitter_description', 'twitter', False)
df = makeFlag(df, "politician_email", 'email', True)

<ipython-input-14-c281889977c6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{delete_keyword}_flag'] = image_flag
<ipython-input-14-c281889977c6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{delete_keyword}_flag'] = image_flag
<ipython-input-14-c281889977c6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [ ]:
df = df[all_we_need]

In [ ]:
def remove_minority(df, col, num):
  for i , k in dict(df[col].value_counts()).items():
    if k <= num:
      df = df.replace(to_replace=i, value="Others")
  return df

df = remove_minority(df, "political_party", 15)
df['political_party'] = df['political_party'].fillna('Others')

In [ ]:
# one_hot_list = ['political_party']

In [ ]:
def one_hot_add(df, category):
  encoder = OneHotEncoder()
  encoded_data = encoder.fit_transform(df[[category]])
  return pd.get_dummies(df, columns=[category], prefix=[category])

# for col in one_hot_list:
#   df = one_hot_add(df, col)

In [ ]:
def pca(df, comp):
  pca = PCA(n_components=comp)
  pca.fit(df)
  return pca.transform(df)

In [ ]:
nltk.download('punkt')
ps = PorterStemmer()

def stemming(sentence):
  words = word_tokenize(sentence)
  output = ""
  for w in words:
    output += " "
    output += ps.stem(w)
  return output

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def tfidf(df, col):
  sentences = list(df[col].fillna(""))
  sentences = [stemming(i) for i in sentences]
  vectorizer = TfidfVectorizer(stop_words='english', max_features=100)
  tfidf_matrix = vectorizer.fit_transform(sentences)
  return pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

scrape -> texual data

In [ ]:
# def getText(link):

#   url = link

#   if link == None:
#     print("empty")
#     return ""

#   try:
#     page = urlopen(url, timeout=3)
#     html = page.read().decode("utf-8")
#     soup = BeautifulSoup(html, "html.parser")
#     return soup.get_text()

#   except:
#     return ""

# tqdm.pandas()
# df['website'] = df['politician_url'].progress_apply(lambda x : getText(x))

14 min for 3300 rows

In [ ]:
df = df.reset_index(drop=True)
for col in ["twitter_description"]:
  tfidf(df, col)
  df[col] = pca(tfidf(df, col), 1)

In [ ]:
needed = [i for i in df.columns if i not in ['we_vote_id',	'political_party',	'politician_url'	,'state_code']]

In [ ]:
df

,we_vote_id,political_party,politician_url,state_code,twitter_followers_count,twitter_description,image_flag,facebook_flag,google_civic_flag,phone_flag,lon,lat
0,wv87pol78432,non_partisan,,al,0,-0.047248,1,1,1,1,-86.9023,32.31820
1,wv87pol13512,,,al,0,-0.047248,0,1,1,1,-86.9023,32.31820
2,wv87pol50829,republican,http://www.tallaco.com/sheriff.asp,al,854,0.084814,1,1,1,1,-86.9023,32.31820
3,wv87pol13510,,,al,0,-0.047248,0,1,1,1,-86.9023,32.31820
4,wv87pol51041,republican,http://co.morgan.al.us/documents/minutes/minut...,al,0,-0.047248,0,0,1,1,-86.9023,32.31820
...,...,...,...,...,...,...,...,...,...,...,...,...
79199,wv87pol34660,democrat,,wy,0,-0.047248,1,0,0,0,-107.2903,43.07597
79200,wv87pol34118,republican,https://zerba.com,wy,0,-0.047248,0,1,1,1,-107.2903,43.07597
79201,wv87pol60821,republican,None,wy,0,-0.047248,0,0,1,0,-107.2903,43.07597
79202,wv87pol34206,republican,https://www.zwonitzerforsenate.com,wy,0,-0.047248,1,1,1,1,-107.2903,43.07597


In [ ]:
def addCluster(df, k=10):
  kmeans = KMeans(n_clusters=k)
  kmeans.fit(df)
  df['Cluster'] = kmeans.labels_
  return df

In [ ]:
df = addCluster(df[needed], 100)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
df['Cluster'].value_counts()

67    67956
0      2789
95     1988
71     1641
47     1243
81      832
33      568
72      321
96      259
53      186
88      185
25      132
80      119
40      106
56       92
84       87
86       81
66       56
14       53
94       40
49       38
73       26
91       26
31       26
83       25
20       22
74       22
11       17
85       17
57       15
62       14
70       14
43       14
23       10
92       10
93        9
63        9
41        8
58        8
15        8
99        7
59        6
69        6
52        5
90        5
77        5
60        4
75        4
18        4
82        4
6         4
76        4
36        4
29        4
32        3
39        3
10        3
35        3
79        2
24        2
44        2
28        2
51        2
55        2
48        2
45        2
26        2
89        2
68        2
5         2
30        1
42        1
54        1
46        1
34        1
65        1
16        1
13        1
3         1
17        1
64        1
8         1
22        1
9   

Number of Politician is 79207

In [ ]:
np.mean(df['Cluster'].value_counts() <= 3)

0.46

rule-based models (same state, same party, if number is still larger than 5 then use twitter, image_flag, facebook_flag)

In [ ]:
df[df['Cluster'] == 67]

,twitter_followers_count,twitter_description,image_flag,facebook_flag,google_civic_flag,phone_flag,lon,lat,Cluster
0,0,-0.047248,1,1,1,1,-86.9023,32.31820,67
1,0,-0.047248,0,1,1,1,-86.9023,32.31820,67
3,0,-0.047248,0,1,1,1,-86.9023,32.31820,67
4,0,-0.047248,0,0,1,1,-86.9023,32.31820,67
6,0,-0.047248,0,1,1,1,-86.9023,32.31820,67
...,...,...,...,...,...,...,...,...,...
79199,0,-0.047248,1,0,0,0,-107.2903,43.07597,67
79200,0,-0.047248,0,1,1,1,-107.2903,43.07597,67
79201,0,-0.047248,0,0,1,0,-107.2903,43.07597,67
79202,0,-0.047248,1,1,1,1,-107.2903,43.07597,67


In [ ]:
df.lat.value_counts()

31.96860    6416
35.75960    5585
37.43160    4232
36.77830    3309
40.71280    3250
41.20330    2748
40.41730    2477
40.26720    2363
32.16560    2346
40.63310    2267
32.35470    2225
44.31480    2146
38.57390    2072
40.05830    1933
27.99440    1905
37.83930    1728
35.51750    1657
47.75110    1636
43.19390    1491
46.72960    1457
39.01190    1451
39.55010    1316
43.78440    1265
41.87800    1205
39.04580    1149
33.44840    1078
34.05220    1028
41.49250    1021
38.59760    1014
33.83610    1004
39.32000     992
44.06820     986
35.00780     985
32.31820     954
46.87970     929
44.94260     883
43.96950     858
45.25380     828
38.80260     820
41.60320     803
47.55150     798
34.97270     730
42.40720     666
30.98430     654
44.55880     595
43.07597     506
21.30690     443
41.58010     409
61.01600     341
38.91080     250
Name: lat, dtype: int64